In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
confirmed_df = pd.read_csv('/kaggle/input/covid19-data-from-john-hopkins-university/RAW_global_confirmed_cases.csv')
death_df = pd.read_csv('/kaggle/input/covid19-data-from-john-hopkins-university/RAW_global_deaths.csv')

In [ ]:
confirmed_df.info()
confirmed_df.columns

In [ ]:
print(death_df.info())
print(death_df.head(30))

In [ ]:
filter = confirmed_df['Country/Region'] == 'Philippines'
ph_confirmed_df = confirmed_df.loc[filter, :].drop(columns=['Country/Region', 'Province/State', 'Lat', 'Long']).T.reset_index()
ph_confirmed_df = ph_confirmed_df.rename(columns={ph_confirmed_df.columns[0]:'Date', ph_confirmed_df.columns[1]:'Cases'})
ph_confirmed_df['Date'] = ph_confirmed_df['Date'].apply(pd.to_datetime)
ph_confirmed_df.tail()


In [ ]:
filter = death_df['Country/Region'] == 'Philippines'
ph_death_df = death_df.loc[filter, :].drop(columns=['Country/Region', 'Province/State', 'Lat', 'Long']).T.reset_index()

ph_death_df = ph_death_df.rename(columns={ph_death_df.columns[0]:'Date', ph_death_df.columns[1]:'Deaths'})
ph_death_df['Date'] = ph_death_df['Date'].apply(pd.to_datetime)
ph_death_df.tail()

In [ ]:
# Merge the 2 dataset
ph_final_df = pd.merge(ph_confirmed_df,ph_death_df, on='Date')
ph_diff = ph_final_df.diff()
ph_final_df['Daily_Cases'] = ph_diff['Cases']
ph_final_df['Daily_Deaths'] = ph_diff['Deaths']
ph_final_df

In [ ]:
sns.set(rc={'figure.figsize':(30,20)})
fig, axs = plt.subplots(2, 2)

sns.lineplot(data=ph_final_df, x='Date', y='Cases', ax=axs[0,0], label='Accumulated Cases')
sns.lineplot(data=ph_final_df, x='Date', y='Deaths',ax=axs[0,0], label='Accumulated Deaths')
axs[0,0].set_title('Philippines Daily Accumulated Covid Cases')
axs[0,0].set_ylabel('Total Number of Cases')
axs[0,0].xaxis.set_major_locator(plt.MaxNLocator(5))
axs[0,0].yaxis.set_major_locator(plt.MaxNLocator(5))

sns.lineplot(data=ph_final_df, x='Date', y='Daily_Cases',ax=axs[0,1],  label='Daily Cases')
sns.lineplot(data=ph_final_df, x='Date', y='Daily_Deaths',ax=axs[0,1], label='Daily Deaths')
axs[0,1].set_title('Philippines Daily Covid Cases')
axs[0,1].set_ylabel('Total Number of Cases')
axs[0,1].xaxis.set_major_locator(plt.MaxNLocator(5))
axs[0,1].yaxis.set_major_locator(plt.MaxNLocator(5))

sns.heatmap(ph_final_df.loc[:, ['Daily_Cases', 'Daily_Deaths']].corr(), ax=axs[1,0], annot=True, annot_kws={'size':12})
axs[1,0].set_title('Correlation between Daily Cases and Daily Deaths')

sns.lineplot(data=ph_final_df.loc[:, ['Daily_Cases']].rolling(7).mean(),ax=axs[1,1], x=ph_final_df.index, y='Daily_Cases', label='7 Day Moving Averages')
sns.lineplot(data=ph_final_df.loc[:, ['Daily_Cases']].rolling(15).mean(),ax=axs[1,1], x=ph_final_df.index, y='Daily_Cases', label='15 Day Moving Averages')
sns.lineplot(data=ph_final_df.loc[:, ['Daily_Cases']].rolling(30).mean(),ax=axs[1,1],x=ph_final_df.index, y='Daily_Cases', label='30 Day Moving Averages')
sns.lineplot(data=ph_final_df.loc[:, ['Daily_Cases']].rolling(90).mean(),ax=axs[1,1],x=ph_final_df.index, y='Daily_Cases', label='90 Day Moving Averages')
axs[1,1].set_title('Daily Cases Moving Averages')